In [1]:
import os, sys
import traceback
import pandas as pd
import re
import glob
import time
from string import ascii_uppercase
from natsort import natsorted
import math
import shutil



print('\t',os.getcwd(), '\n')

def write_to_subfolder(result_df, new_file, subdirectory):
    
    curr_dir = os.getcwd()
    new_dir = os.path.join(curr_dir,subdirectory)
    counter = 0
    
    if curr_dir.find(subdirectory) == -1:
        
        try:
            os.mkdir(subdirectory)
        except Exception:
            pass
    
        os.chdir(new_dir)
        print('File written  :-  ', new_file)
        result_df.to_csv(new_file, index = False)
        counter = 1
        os.chdir('..')
        
    else:
        os.chdir(new_dir)
        print('File written  :-  ', new_file)
        result_df.to_csv(new_file, index = True)
        os.chdir('..')
        
        
        
def Read_Primer_Guide_Files ():
        
    IDT_order = glob.glob("**/*Reformatted_IDT_SDM*.csv")

    for filename in IDT_order:
        Dilution_picklist_Fluent(filename)
        
    if not filename:
        print('Verify if IDT_Primers_ and IDT_Guides_ files exists in subfolder!!!')

                
def Dilution_picklist_Fluent(filename):
        
    print('file read:- ', filename, '\n')
    df = pd.read_csv(filename)  
                
    column_names = ["Source plate", "S-Well position", "Destination plate", "D-Well position", "Volume", "Primer_name"]
    df_picklist_lyophilized = pd.DataFrame(columns = column_names)
    df_picklist_54ulwater = pd.DataFrame(columns = column_names)
    df_picklist_6ulprimer = pd.DataFrame(columns = column_names)
    
    Multiplier = 5
        
    for index, row in df.iterrows():
        name = df['Sequence Name'].loc[index]
        nmol = df['nmoles'].loc[index]
        temp = re.findall('[0-9][0-9].[0-9]', str(nmol))
        if not temp:
            temp = re.findall('[0-9][0-9]', nmol)
            
        temp = math.ceil(float(temp[0]))
        h20_vol = int((temp * Multiplier))
        Iter = math.ceil(h20_vol / 120)
        
        D_well = df['Well Position'].loc[index]
                  
        # Add water to lyophilized primers
        S_plate = 'Water_300mltrough'
        S_well = 'A1'
        D_plate = 'IDT_primers_001'
        
        for x in range(Iter):
            if h20_vol > 120:
                h20_vol_new = 120
            else:
                h20_vol_new = h20_vol
                
            if x == (Iter - 1):
                h20_vol_new = h20_vol - (120*x)
                if h20_vol_new < 11:
                    continue
                
            list1 = [S_plate, S_well, D_plate, D_well, h20_vol_new, name]
            a_series = pd.Series(list1, index = df_picklist_lyophilized.columns)
            df_picklist_lyophilized = df_picklist_lyophilized.append(a_series, ignore_index=True)
            list1.clear()

        # Add 54ul to corning 384 well plate
        S_plate = 'Water_300mltrough'
        S_well = 'A1'
        D_plate = 'Primers_10uM_001'
        list1 = [S_plate, S_well, D_plate, D_well, '56', name]
        a_series = pd.Series(list1, index = df_picklist_54ulwater.columns)
        df_picklist_54ulwater = df_picklist_54ulwater.append(a_series, ignore_index=True)
        list1.clear()
        
        # Transfer 6ul to primer from IDT plate to corning 384 well plate
        S_plate = 'IDT_primers_001'
        S_well = D_well
        D_plate = 'Primers_10uM_001'
        list1 = [S_plate, S_well, D_plate, D_well, '7', name]
        a_series = pd.Series(list1, index = df_picklist_6ulprimer.columns)
        df_picklist_6ulprimer = df_picklist_6ulprimer.append(a_series, ignore_index=True)
        list1.clear()
        
        
    Filename_1  = '001_IDT_primers_hydration_worklist_1.csv'   
    Filename_2  = '002_Add_54ul_water_worklist_2.csv'   
    Filename_3  = '003_Transfer_6ul_primer_worklist_3.csv'   

    write_to_subfolder (df_picklist_lyophilized, Filename_1, 'Dilution_Picklist')
    write_to_subfolder (df_picklist_54ulwater, Filename_2, 'Dilution_Picklist')
    write_to_subfolder (df_picklist_6ulprimer, Filename_3, 'Dilution_Picklist')

    CSV_to_GWL_add_water(Filename_1, df_picklist_lyophilized, 'F;\n')
    CSV_to_GWL_add_water(Filename_2, df_picklist_54ulwater, 'F;\n')
    CSV_to_GWL_add_water(Filename_3, df_picklist_6ulprimer, 'W;\n')


def CSV_to_GWL_add_water(Filename, Fluent_picklist, transfer_mode):
    
    Step_list =[]

    for index, row in Fluent_picklist.iterrows():

        Volume = Fluent_picklist['Volume'].loc[index]

        S_plate = Fluent_picklist['Source plate'].loc[index]
        S_well = Fluent_picklist['S-Well position'].loc[index]
        Step_1 = str('A;'+ S_plate+';;;'+str(S_well)+';;'+str(Volume)+';;;;\n')
        Step_list.append(Step_1)

        D_plate = Fluent_picklist['Destination plate'].loc[index]
        D_well = Fluent_picklist['D-Well position'].loc[index]
        Step_2 = str('D;'+ D_plate+';;;'+str(D_well)+';;'+str(Volume)+';;;;\n')
        Step_list.append(Step_2)
        
        Step_list.append(transfer_mode)

    Gwl_Filename = Filename[:-4]+'.gwl'
    
    F = open(Gwl_Filename, "w+")
    F.writelines(Step_list)
    F.close()
    
    
def Move_GWL_files_to_subfolder():
    
    source = os.getcwd()
    Dest_dir = os.path.join(source, 'gwl_worklists')
    print('\n Moving all gwl flies to:  \t', Dest_dir, '\n')

    if not os.path.exists(Dest_dir):
        #shutil.rmtree(Dest_dir)
        os.makedirs(Dest_dir)

    gwl_files = glob.glob('*.gwl')

    for file in gwl_files:
        file_name_source = os.path.join(source, file)
        file_name_dest = os.path.join(Dest_dir, file)
        
        shutil.move(file_name_source, file_name_dest)

    
if __name__ == '__main__':
    
    Read_Primer_Guide_Files()
    Move_GWL_files_to_subfolder()
    


	 /Users/nilmani/Desktop/Protein_Engineering 

file read:-  Generated_file/Reformatted_IDT_SDM.csv 

File written  :-   001_IDT_primers_hydration_worklist_1.csv
File written  :-   002_Add_54ul_water_worklist_2.csv
File written  :-   003_Transfer_6ul_primer_worklist_3.csv

 Moving all gwl flies to:  	 /Users/nilmani/Desktop/Protein_Engineering/gwl_worklists 

